# Day 6: データ型と欠損値処理 - ベーシック編

## 今日の目標
- 高度な欠損値処理手法を習得する
- 欠損値パターンの分析方法を学ぶ
- 実際のデータセットを使った実践的な処理を行う
- パフォーマンスを考慮したデータ型最適化

## アジェンダ
1. 高度な欠損値補完手法
2. 欠損値パターンの分析
3. 時系列データの欠損値処理
4. 機械学習による欠損値補完
5. パフォーマンス最適化
6. 実践的なケーススタディ

## 1. 必要なライブラリの読み込み

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("ライブラリの読み込み完了！")

ライブラリの読み込み完了！


## 2. 高度な欠損値補完手法

### 2.1 K近傍法による補完（KNN Imputation）

In [7]:
# サンプルデータの作成（不動産データ）
np.random.seed(42)
n_properties = 200

# 相関のあるデータを生成
area = np.random.normal(80, 30, n_properties)  # 面積 (m²)
rooms = np.clip(np.round(area / 25 + np.random.normal(0, 0.5, n_properties)), 1, 6)  # 部屋数
age = np.random.exponential(10, n_properties)  # 築年数
price = (area * 50 + rooms * 5 - age * 2 + np.random.normal(0, 10, n_properties)) * 10000  # 価格

real_estate_data = pd.DataFrame({
    '面積': area,
    '部屋数': rooms,
    '築年数': age,
    '価格': price,
    '最寄り駅距離': np.random.gamma(2, 3, n_properties),  # 分
    '階数': np.random.randint(1, 15, n_properties)
})

# 欠損値を意図的に作成（MCAR: Missing Completely At Random）
missing_indices = np.random.choice(n_properties, size=int(n_properties * 0.15), replace=False)
real_estate_data.loc[missing_indices, '価格'] = np.nan

missing_indices = np.random.choice(n_properties, size=int(n_properties * 0.1), replace=False)
real_estate_data.loc[missing_indices, '面積'] = np.nan

missing_indices = np.random.choice(n_properties, size=int(n_properties * 0.08), replace=False)
real_estate_data.loc[missing_indices, '最寄り駅距離'] = np.nan

print("不動産データ（最初の10行）:")
print(real_estate_data.head(10))
print("\n欠損値の状況:")
print(real_estate_data.isnull().sum())
print("\n相関係数:")
print(real_estate_data.corr())

不動産データ（最初の10行）:
        面積  部屋数     築年数         価格  最寄り駅距離  階数
0   94.901  4.0   5.358  4.758e+07   2.151  14
1   75.852  3.0   3.194  3.797e+07   2.358  13
2      NaN  5.0   0.580  4.990e+07   8.232   4
3  125.691  6.0  20.004  6.270e+07   3.086   2
4   72.975  2.0  16.761  3.629e+07   5.488  10
5   72.976  2.0  81.724  3.491e+07  10.903  12
6  127.376  5.0  56.949  6.283e+07   2.741   7
7  103.023  4.0   8.107  5.176e+07   4.866   8
8   65.916  3.0  14.653  3.290e+07  15.472   2
9   96.277  6.0  28.962  4.783e+07   8.273   5

欠損値の状況:
面積        20
部屋数        0
築年数        0
価格        30
最寄り駅距離    16
階数         0
dtype: int64

相関係数:
           面積    部屋数    築年数     価格  最寄り駅距離     階数
面積      1.000  0.884  0.087  1.000  -0.041  0.062
部屋数     0.884  1.000  0.048  0.890  -0.070 -0.002
築年数     0.087  0.048  1.000  0.040  -0.020  0.120
価格      1.000  0.890  0.040  1.000  -0.016  0.056
最寄り駅距離 -0.041 -0.070 -0.020 -0.016   1.000 -0.036
階数      0.062 -0.002  0.120  0.056  -0.036  1.000


In [8]:
# KNN補完の実行
from sklearn.impute import KNNImputer

# KNN補完器の設定（k=5を使用）
knn_imputer = KNNImputer(n_neighbors=5)

# データをコピーして補完
real_estate_knn = real_estate_data.copy()
real_estate_knn_filled = pd.DataFrame(
    knn_imputer.fit_transform(real_estate_knn),
    columns=real_estate_knn.columns
)

print("KNN補完後の欠損値数:")
print(real_estate_knn_filled.isnull().sum())

# 補完前後の統計比較
print("\n価格の統計（補完前後比較）:")
print("元データ（欠損値除く）:")
print(real_estate_data['価格'].describe())
print("\nKNN補完後:")
print(real_estate_knn_filled['価格'].describe())

KNN補完後の欠損値数:
面積        0
部屋数       0
築年数       0
価格        0
最寄り駅距離    0
階数        0
dtype: int64

価格の統計（補完前後比較）:
元データ（欠損値除く）:
count    1.700e+02
mean     3.979e+07
std      1.404e+07
min      1.007e+07
25%      2.922e+07
50%      4.054e+07
75%      4.751e+07
max      8.001e+07
Name: 価格, dtype: float64

KNN補完後:
count    2.000e+02
mean     3.966e+07
std      1.329e+07
min      1.007e+07
25%      3.112e+07
50%      4.019e+07
75%      4.705e+07
max      8.001e+07
Name: 価格, dtype: float64


### 2.2 回帰による補完（Iterative Imputation）

In [ ]:
# 回帰による補完
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 回帰補完器の設定
iterative_imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=10, random_state=42),
    random_state=42,
    max_iter=10
)

# 回帰補完の実行
real_estate_iterative = real_estate_data.copy()
real_estate_iterative_filled = pd.DataFrame(
    iterative_imputer.fit_transform(real_estate_iterative),
    columns=real_estate_iterative.columns
)

print("回帰補完後の欠損値数:")
print(real_estate_iterative_filled.isnull().sum())

print("\n価格の統計（回帰補完）:")
print(real_estate_iterative_filled['価格'].describe())

### 2.3 補完手法の比較

In [ ]:
# 異なる補完手法の比較
# 1. 平均値補完
real_estate_mean = real_estate_data.copy()
for col in real_estate_mean.select_dtypes(include=[np.number]).columns:
    real_estate_mean[col] = real_estate_mean[col].fillna(real_estate_mean[col].mean())

# 2. 中央値補完
real_estate_median = real_estate_data.copy()
for col in real_estate_median.select_dtypes(include=[np.number]).columns:
    real_estate_median[col] = real_estate_median[col].fillna(real_estate_median[col].median())

# 補完手法の比較可視化
plt.figure(figsize=(15, 10))

# 価格の分布比較
plt.subplot(2, 3, 1)
plt.hist(real_estate_data['価格'].dropna(), bins=20, alpha=0.7, label='元データ', density=True)
plt.hist(real_estate_mean['価格'], bins=20, alpha=0.7, label='平均値補完', density=True)
plt.title('価格分布: 平均値補完')
plt.legend()

plt.subplot(2, 3, 2)
plt.hist(real_estate_data['価格'].dropna(), bins=20, alpha=0.7, label='元データ', density=True)
plt.hist(real_estate_median['価格'], bins=20, alpha=0.7, label='中央値補完', density=True)
plt.title('価格分布: 中央値補完')
plt.legend()

plt.subplot(2, 3, 3)
plt.hist(real_estate_data['価格'].dropna(), bins=20, alpha=0.7, label='元データ', density=True)
plt.hist(real_estate_knn_filled['価格'], bins=20, alpha=0.7, label='KNN補完', density=True)
plt.title('価格分布: KNN補完')
plt.legend()

plt.subplot(2, 3, 4)
plt.hist(real_estate_data['価格'].dropna(), bins=20, alpha=0.7, label='元データ', density=True)
plt.hist(real_estate_iterative_filled['価格'], bins=20, alpha=0.7, label='回帰補完', density=True)
plt.title('価格分布: 回帰補完')
plt.legend()

# 相関係数の比較
plt.subplot(2, 3, 5)
methods = ['元データ', '平均値', '中央値', 'KNN', '回帰']
correlations = [
    real_estate_data.corr().loc['価格', '面積'],
    real_estate_mean.corr().loc['価格', '面積'],
    real_estate_median.corr().loc['価格', '面積'],
    real_estate_knn_filled.corr().loc['価格', '面積'],
    real_estate_iterative_filled.corr().loc['価格', '面積']
]

plt.bar(methods, correlations)
plt.title('価格と面積の相関係数比較')
plt.xticks(rotation=45)
plt.ylabel('相関係数')

# 統計値の比較表
plt.subplot(2, 3, 6)
stats_comparison = pd.DataFrame({
    '平均値補完': real_estate_mean['価格'].describe(),
    '中央値補完': real_estate_median['価格'].describe(),
    'KNN補完': real_estate_knn_filled['価格'].describe(),
    '回帰補完': real_estate_iterative_filled['価格'].describe()
})

# 統計値の比較をテキストで表示
plt.text(0.1, 0.5, stats_comparison.round(2).to_string(), 
         transform=plt.gca().transAxes, fontsize=8, verticalalignment='center')
plt.title('統計値比較')
plt.axis('off')

plt.tight_layout()
plt.show()

print("\n補完手法別の統計比較:")
print(stats_comparison.round(2))

## 3. 欠損値パターンの分析

### 3.1 欠損値の種類

In [ ]:
# 異なる種類の欠損値パターンを作成
np.random.seed(42)
n_samples = 1000

# ベースデータの作成
income = np.random.lognormal(mean=6, sigma=0.5, size=n_samples)  # 年収（万円）
age = np.random.randint(22, 65, n_samples)  # 年齢
education_years = np.random.randint(12, 20, n_samples)  # 教育年数
satisfaction = np.random.randint(1, 11, n_samples)  # 満足度 (1-10)

missing_patterns_data = pd.DataFrame({
    '年収': income,
    '年齢': age,
    '教育年数': education_years,
    '満足度': satisfaction
})

# MCAR (Missing Completely At Random): 完全にランダムな欠損
mcar_indices = np.random.choice(n_samples, size=int(n_samples * 0.1), replace=False)
missing_patterns_data.loc[mcar_indices, '満足度'] = np.nan

# MAR (Missing At Random): 他の観測値に依存する欠損
# 高年収者は年収を回答しない傾向があると仮定
high_income_mask = missing_patterns_data['年収'] > missing_patterns_data['年収'].quantile(0.8)
mar_indices = missing_patterns_data[high_income_mask].sample(frac=0.3).index
missing_patterns_data.loc[mar_indices, '年収'] = np.nan

# MNAR (Missing Not At Random): 欠損値自体が値に依存
# 教育年数が低い人は教育年数を回答しない傾向があると仮定
low_education_mask = missing_patterns_data['教育年数'] < 15
mnar_indices = missing_patterns_data[low_education_mask].sample(frac=0.2).index
missing_patterns_data.loc[mnar_indices, '教育年数'] = np.nan

print("欠損値パターンの分析データ:")
print(missing_patterns_data.isnull().sum())
print("\n欠損値割合:")
print((missing_patterns_data.isnull().sum() / len(missing_patterns_data) * 100).round(2))

### 3.2 欠損値パターンの可視化

In [ ]:
# 欠損値パターンの詳細分析
plt.figure(figsize=(15, 10))

# 1. 欠損値マトリックス
plt.subplot(2, 3, 1)
sns.heatmap(missing_patterns_data.isnull(), cbar=True, yticklabels=False)
plt.title('欠損値パターンマトリックス')

# 2. 欠損値の共起パターン
plt.subplot(2, 3, 2)
missing_combinations = missing_patterns_data.isnull().groupby(
    missing_patterns_data.isnull().apply(tuple, axis=1)
).size().sort_values(ascending=False)

top_patterns = missing_combinations.head(10)
pattern_labels = [str(pattern) for pattern in top_patterns.index]
plt.bar(range(len(top_patterns)), top_patterns.values)
plt.title('欠損値パターンの頻度')
plt.xlabel('パターン')
plt.ylabel('頻度')
plt.xticks(range(len(top_patterns)), 
           [f'パターン{i+1}' for i in range(len(top_patterns))], 
           rotation=45)

# 3. 年収の欠損パターン分析（MAR分析）
plt.subplot(2, 3, 3)
income_complete = missing_patterns_data.dropna(subset=['年収'])['年収']
all_ages = missing_patterns_data['年齢']
ages_with_income = missing_patterns_data.dropna(subset=['年収'])['年齢']
ages_without_income = missing_patterns_data[missing_patterns_data['年収'].isnull()]['年齢']

plt.hist(ages_with_income, bins=20, alpha=0.7, label='年収回答あり', density=True)
plt.hist(ages_without_income, bins=20, alpha=0.7, label='年収回答なし', density=True)
plt.title('年収回答状況別の年齢分布')
plt.xlabel('年齢')
plt.ylabel('密度')
plt.legend()

# 4. 教育年数の欠損パターン分析（MNAR分析）
plt.subplot(2, 3, 4)
edu_complete = missing_patterns_data.dropna(subset=['教育年数'])['教育年数']
edu_missing_ages = missing_patterns_data[missing_patterns_data['教育年数'].isnull()]['年齢']
edu_complete_ages = missing_patterns_data.dropna(subset=['教育年数'])['年齢']

plt.hist(edu_complete_ages, bins=15, alpha=0.7, label='教育年数回答あり', density=True)
plt.hist(edu_missing_ages, bins=15, alpha=0.7, label='教育年数回答なし', density=True)
plt.title('教育年数回答状況別の年齢分布')
plt.xlabel('年齢')
plt.ylabel('密度')
plt.legend()

# 5. 相関分析：欠損値と他の変数の関係
plt.subplot(2, 3, 5)
# 欠損値フラグを作成
missing_flags = missing_patterns_data.isnull().astype(int)
missing_flags.columns = [f'{col}_欠損' for col in missing_flags.columns]

# 数値データと欠損フラグの相関
combined_data = pd.concat([
    missing_patterns_data.select_dtypes(include=[np.number]),
    missing_flags
], axis=1)

correlation_with_missing = combined_data.corr()
missing_corr = correlation_with_missing.loc[
    combined_data.select_dtypes(include=[np.number]).columns[:4],
    missing_flags.columns
]

sns.heatmap(missing_corr, annot=True, cmap='coolwarm', center=0)
plt.title('変数と欠損パターンの相関')

# 6. Little's MCAR test の結果（疑似）
plt.subplot(2, 3, 6)
mcar_results = pd.DataFrame({
    '変数': ['満足度', '年収', '教育年数'],
    'MCAR_p値': [0.456, 0.023, 0.012],  # 疑似的な値
    '欠損タイプ': ['MCAR', 'MAR', 'MNAR']
})

colors = ['green' if p > 0.05 else 'red' for p in mcar_results['MCAR_p値']]
plt.bar(mcar_results['変数'], mcar_results['MCAR_p値'], color=colors, alpha=0.7)
plt.axhline(y=0.05, color='red', linestyle='--', label='p=0.05')
plt.title('欠損値ランダム性テスト')
plt.xlabel('変数')
plt.ylabel('p値')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("\n欠損値パターンの分析結果:")
print("1. 満足度: MCAR（完全にランダム）- p値が0.05以上")
print("2. 年収: MAR（他の変数に依存）- 高年収者が回答を避ける傾向")
print("3. 教育年数: MNAR（値自体に依存）- 低教育年数者が回答を避ける傾向")

## 4. 時系列データの欠損値処理

In [ ]:
# 時系列データの作成（センサーデータを模擬）
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=365, freq='D')

# トレンドと季節性を持つデータ
trend = np.linspace(100, 120, 365)
seasonal = 10 * np.sin(2 * np.pi * np.arange(365) / 365.25)
noise = np.random.normal(0, 2, 365)
temperature = trend + seasonal + noise

# 湿度（温度と逆相関）
humidity = 80 - 0.3 * (temperature - 100) + np.random.normal(0, 3, 365)

# 気圧
pressure = 1013 + np.random.normal(0, 10, 365)

timeseries_data = pd.DataFrame({
    '日付': dates,
    '温度': temperature,
    '湿度': humidity,
    '気圧': pressure
})

timeseries_data.set_index('日付', inplace=True)

# 時系列的な欠損パターンを作成
# 1. ランダムな単発欠損
random_missing = np.random.choice(365, size=20, replace=False)
timeseries_data.loc[timeseries_data.index[random_missing], '温度'] = np.nan

# 2. 連続する欠損（センサー故障を模擬）
consecutive_start = 100
timeseries_data.loc[timeseries_data.index[consecutive_start:consecutive_start+7], '湿度'] = np.nan

# 3. 周期的な欠損（メンテナンス期間を模擬）
maintenance_days = range(0, 365, 30)  # 30日ごと
timeseries_data.loc[timeseries_data.index[maintenance_days], '気圧'] = np.nan

print("時系列データ（最初の10日）:")
print(timeseries_data.head(10))
print("\n欠損値の状況:")
print(timeseries_data.isnull().sum())

### 4.1 時系列データの欠損値補完手法

In [ ]:
# 時系列データの様々な補完手法
ts_methods = {}

# 1. 前方補完（Forward Fill）
ts_methods['前方補完'] = timeseries_data.fillna(method='ffill')

# 2. 後方補完（Backward Fill）
ts_methods['後方補完'] = timeseries_data.fillna(method='bfill')

# 3. 線形補間
ts_methods['線形補間'] = timeseries_data.interpolate(method='linear')

# 4. 時系列補間
ts_methods['時間補間'] = timeseries_data.interpolate(method='time')

# 5. スプライン補間
ts_methods['スプライン補間'] = timeseries_data.interpolate(method='spline', order=3)

# 6. 移動平均補完
ts_rolling = timeseries_data.copy()
for col in ts_rolling.columns:
    rolling_mean = ts_rolling[col].rolling(window=7, center=True).mean()
    ts_rolling[col] = ts_rolling[col].fillna(rolling_mean)
ts_methods['移動平均補完'] = ts_rolling.fillna(method='ffill').fillna(method='bfill')

# 結果の可視化
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.ravel()

colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown']
method_names = list(ts_methods.keys())

for i, (method_name, data) in enumerate(ts_methods.items()):
    ax = axes[i]
    
    # 温度データをプロット（1月のみ）
    jan_data = data['2024-01-01':'2024-01-31']
    original_jan = timeseries_data['2024-01-01':'2024-01-31']
    
    ax.plot(jan_data.index, jan_data['温度'], color=colors[i], label=method_name, linewidth=2)
    
    # 欠損値の位置をマーク
    missing_mask = original_jan['温度'].isnull()
    if missing_mask.any():
        ax.scatter(original_jan[missing_mask].index, 
                  jan_data[missing_mask]['温度'], 
                  color='red', s=50, zorder=5, label='補完値')
    
    ax.set_title(f'{method_name}による補完')
    ax.set_ylabel('温度')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 補完手法の評価
print("\n補完手法の統計比較（温度）:")
temp_stats = pd.DataFrame({
    method: data['温度'].describe() 
    for method, data in ts_methods.items()
})
print(temp_stats.round(2))

### 4.2 季節性を考慮した補完

In [ ]:
# 季節性を考慮した高度な補完
from scipy import stats

def seasonal_imputation(series, period=7):
    """季節性を考慮した欠損値補完"""
    result = series.copy()
    
    for i in range(len(series)):
        if pd.isna(series.iloc[i]):
            # 同じ曜日（または周期）のデータを取得
            same_period_indices = range(i % period, len(series), period)
            same_period_values = series.iloc[same_period_indices].dropna()
            
            if len(same_period_values) > 0:
                # 近傍の値の重み付き平均を使用
                weights = np.exp(-np.abs(np.array(same_period_indices) - i) / 30)
                weights = weights[:len(same_period_values)]
                
                result.iloc[i] = np.average(same_period_values, weights=weights)
            else:
                # フォールバック: 線形補間
                result.iloc[i] = series.interpolate().iloc[i]
    
    return result

# 季節性補完の適用
ts_seasonal = timeseries_data.copy()
for col in ts_seasonal.columns:
    ts_seasonal[col] = seasonal_imputation(ts_seasonal[col], period=7)

# 結果の比較
plt.figure(figsize=(15, 8))

# 温度データの比較（2月のデータ）
feb_original = timeseries_data['2024-02-01':'2024-02-29']
feb_linear = ts_methods['線形補間']['2024-02-01':'2024-02-29']
feb_seasonal = ts_seasonal['2024-02-01':'2024-02-29']

plt.subplot(2, 1, 1)
plt.plot(feb_linear.index, feb_linear['温度'], 'b-', label='線形補間', linewidth=2)
plt.plot(feb_seasonal.index, feb_seasonal['温度'], 'r-', label='季節性補間', linewidth=2)

# 欠損値の位置をマーク
missing_mask = feb_original['温度'].isnull()
if missing_mask.any():
    plt.scatter(feb_original[missing_mask].index, 
               feb_linear[missing_mask]['温度'], 
               color='blue', s=50, zorder=5, label='線形補完値')
    plt.scatter(feb_original[missing_mask].index, 
               feb_seasonal[missing_mask]['温度'], 
               color='red', s=50, zorder=5, label='季節性補完値')

plt.title('補完手法の比較（2月の温度データ）')
plt.ylabel('温度')
plt.legend()
plt.grid(True, alpha=0.3)

# 湿度データの連続欠損補完比較
plt.subplot(2, 1, 2)
apr_period = slice('2024-04-05', '2024-04-20')
plt.plot(timeseries_data.loc[apr_period].index, 
         ts_methods['線形補間'].loc[apr_period]['湿度'], 'b-', 
         label='線形補間', linewidth=2)
plt.plot(timeseries_data.loc[apr_period].index, 
         ts_seasonal.loc[apr_period]['湿度'], 'r-', 
         label='季節性補間', linewidth=2)

# 連続欠損期間をハイライト
consecutive_period = slice('2024-04-10', '2024-04-17')
plt.axvspan(timeseries_data.loc[consecutive_period].index[0], 
           timeseries_data.loc[consecutive_period].index[-1], 
           alpha=0.3, color='yellow', label='連続欠損期間')

plt.title('連続欠損の補完比較（湿度データ）')
plt.ylabel('湿度')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. 機械学習による欠損値補完の評価

In [ ]:
# 補完精度の評価実験
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score

def evaluate_imputation_methods(data, target_column, missing_ratio=0.2, n_splits=5):
    """欠損値補完手法の評価"""
    
    # 完全なデータを準備
    complete_data = data.dropna()
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = []
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(complete_data)):
        # データを分割
        train_data = complete_data.iloc[train_idx].copy()
        test_data = complete_data.iloc[test_idx].copy()
        
        # テストデータに人工的な欠損を作成
        n_missing = int(len(test_data) * missing_ratio)
        missing_idx = np.random.choice(len(test_data), n_missing, replace=False)
        
        # 真の値を保存
        true_values = test_data.iloc[missing_idx][target_column].values
        
        # 欠損を作成
        test_data_missing = test_data.copy()
        test_data_missing.iloc[missing_idx, test_data_missing.columns.get_loc(target_column)] = np.nan
        
        # 様々な補完手法を適用
        methods_performance = {}
        
        # 1. 平均値補完
        mean_filled = test_data_missing.copy()
        mean_filled[target_column] = mean_filled[target_column].fillna(train_data[target_column].mean())
        pred_mean = mean_filled.iloc[missing_idx][target_column].values
        methods_performance['平均値'] = {
            'MAE': mean_absolute_error(true_values, pred_mean),
            'R2': r2_score(true_values, pred_mean)
        }
        
        # 2. KNN補完
        knn_imputer = KNNImputer(n_neighbors=5)
        knn_filled = pd.DataFrame(
            knn_imputer.fit_transform(test_data_missing),
            columns=test_data_missing.columns,
            index=test_data_missing.index
        )
        pred_knn = knn_filled.iloc[missing_idx][target_column].values
        methods_performance['KNN'] = {
            'MAE': mean_absolute_error(true_values, pred_knn),
            'R2': r2_score(true_values, pred_knn)
        }
        
        # 3. 回帰補完
        iterative_imputer = IterativeImputer(
            estimator=RandomForestRegressor(n_estimators=10, random_state=42),
            random_state=42
        )
        iterative_filled = pd.DataFrame(
            iterative_imputer.fit_transform(test_data_missing),
            columns=test_data_missing.columns,
            index=test_data_missing.index
        )
        pred_iterative = iterative_filled.iloc[missing_idx][target_column].values
        methods_performance['回帰'] = {
            'MAE': mean_absolute_error(true_values, pred_iterative),
            'R2': r2_score(true_values, pred_iterative)
        }
        
        results.append(methods_performance)
    
    return results

# 不動産データで評価実験を実行
evaluation_results = evaluate_imputation_methods(
    real_estate_data, '価格', missing_ratio=0.2, n_splits=5
)

# 結果の集計
methods = ['平均値', 'KNN', '回帰']
metrics = ['MAE', 'R2']

summary_results = {}
for method in methods:
    summary_results[method] = {}
    for metric in metrics:
        values = [result[method][metric] for result in evaluation_results]
        summary_results[method][metric] = {
            'mean': np.mean(values),
            'std': np.std(values)
        }

# 結果の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
mae_means = [summary_results[method]['MAE']['mean'] for method in methods]
mae_stds = [summary_results[method]['MAE']['std'] for method in methods]
plt.bar(methods, mae_means, yerr=mae_stds, capsize=5)
plt.title('平均絶対誤差（MAE）の比較')
plt.ylabel('MAE')

plt.subplot(1, 2, 2)
r2_means = [summary_results[method]['R2']['mean'] for method in methods]
r2_stds = [summary_results[method]['R2']['std'] for method in methods]
plt.bar(methods, r2_means, yerr=r2_stds, capsize=5)
plt.title('決定係数（R²）の比較')
plt.ylabel('R²')

plt.tight_layout()
plt.show()

# 数値結果の表示
print("補完手法の評価結果（5-fold交差検証）:")
print("\nMAE（平均絶対誤差）- 小さいほど良い:")
for method in methods:
    mean_mae = summary_results[method]['MAE']['mean']
    std_mae = summary_results[method]['MAE']['std']
    print(f"{method}: {mean_mae:.2f} ± {std_mae:.2f}")

print("\nR²（決定係数）- 大きいほど良い:")
for method in methods:
    mean_r2 = summary_results[method]['R2']['mean']
    std_r2 = summary_results[method]['R2']['std']
    print(f"{method}: {mean_r2:.3f} ± {std_r2:.3f}")

## 6. パフォーマンス最適化

In [ ]:
# 大規模データでのパフォーマンス比較
import time
from memory_profiler import profile

def create_large_dataset(n_rows=10000, n_cols=20, missing_ratio=0.1):
    """大規模データセットの作成"""
    np.random.seed(42)
    
    # データ生成
    data = np.random.randn(n_rows, n_cols)
    
    # 相関のある列を作成
    for i in range(1, n_cols):
        data[:, i] = 0.5 * data[:, i-1] + 0.5 * data[:, i]
    
    df = pd.DataFrame(data, columns=[f'feature_{i}' for i in range(n_cols)])
    
    # ランダムに欠損値を作成
    for col in df.columns:
        missing_mask = np.random.random(n_rows) < missing_ratio
        df.loc[missing_mask, col] = np.nan
    
    return df

# 大規模データセットの作成
large_data = create_large_dataset(n_rows=10000, n_cols=20, missing_ratio=0.1)
print(f"大規模データセット: {large_data.shape}")
print(f"欠損値数: {large_data.isnull().sum().sum()}")
print(f"メモリ使用量: {large_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# パフォーマンス測定
performance_results = {}

# 1. 平均値補完
start_time = time.time()
mean_filled = large_data.fillna(large_data.mean())
mean_time = time.time() - start_time
performance_results['平均値補完'] = mean_time

# 2. KNN補完（小さなk値）
start_time = time.time()
knn_imputer_fast = KNNImputer(n_neighbors=3)
knn_filled_fast = pd.DataFrame(
    knn_imputer_fast.fit_transform(large_data),
    columns=large_data.columns
)
knn_fast_time = time.time() - start_time
performance_results['KNN補完(k=3)'] = knn_fast_time

# 3. 前方補完
start_time = time.time()
ffill_filled = large_data.fillna(method='ffill')
ffill_time = time.time() - start_time
performance_results['前方補完'] = ffill_time

# 4. 線形補間
start_time = time.time()
interpolate_filled = large_data.interpolate()
interpolate_time = time.time() - start_time
performance_results['線形補間'] = interpolate_time

# パフォーマンス結果の可視化
plt.figure(figsize=(10, 6))
methods = list(performance_results.keys())
times = list(performance_results.values())

bars = plt.bar(methods, times)
plt.title('欠損値補完手法のパフォーマンス比較')
plt.ylabel('実行時間（秒）')
plt.xticks(rotation=45)

# 各バーに実行時間を表示
for bar, time_val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{time_val:.2f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nパフォーマンス結果:")
for method, time_val in performance_results.items():
    print(f"{method}: {time_val:.2f}秒")

# メモリ効率的なデータ型の推奨
print("\nメモリ最適化のための推奨事項:")
print("1. カテゴリカルデータ → category型")
print("2. 整数データ → 適切なint型（int8, int16, int32）")
print("3. 浮動小数点 → float32（精度が十分な場合）")
print("4. 大規模データ → チャンク処理またはDaskの利用")

## 7. 実践的なケーススタディ: 顧客購買データの前処理

In [ ]:
# 実際のビジネスシナリオを模擬したデータセット
np.random.seed(42)
n_customers = 5000

# 顧客の基本情報
ages = np.random.randint(18, 80, n_customers)
genders = np.random.choice(['M', 'F', 'Other'], n_customers, p=[0.48, 0.48, 0.04])
income_levels = np.random.choice(['Low', 'Medium', 'High'], n_customers, p=[0.3, 0.5, 0.2])

# 収入（カテゴリに基づく）
income_mapping = {'Low': (200, 400), 'Medium': (400, 800), 'High': (800, 1500)}
incomes = []
for level in income_levels:
    low, high = income_mapping[level]
    incomes.append(np.random.uniform(low, high))

# 購買行動（年齢と収入に依存）
purchase_frequency = (ages / 100 + np.array(incomes) / 1000 + np.random.normal(0, 0.2, n_customers)) * 50
purchase_frequency = np.clip(purchase_frequency, 0, 200)

# 満足度（購買頻度と相関）
satisfaction = 1 + 4 * (purchase_frequency / 200) + np.random.normal(0, 0.5, n_customers)
satisfaction = np.clip(satisfaction, 1, 5)

# 地域
regions = np.random.choice(['北海道', '関東', '関西', '九州', '其他'], n_customers, 
                          p=[0.1, 0.4, 0.25, 0.15, 0.1])

# データフレームの作成
customer_business_data = pd.DataFrame({
    '顧客ID': range(1, n_customers + 1),
    '年齢': ages,
    '性別': genders,
    '年収': incomes,
    '収入レベル': income_levels,
    '購買頻度': purchase_frequency,
    '満足度': satisfaction,
    '地域': regions
})

# 現実的な欠損パターンを作成

# 1. 年収の欠損（プライバシーを理由とした非回答）
# 高年収者と女性の方が回答率が低い傾向
income_missing_prob = 0.15 + 0.1 * (customer_business_data['年収'] > 800) + 0.05 * (customer_business_data['性別'] == 'F')
income_missing_mask = np.random.random(n_customers) < income_missing_prob
customer_business_data.loc[income_missing_mask, '年収'] = np.nan

# 2. 満足度の欠損（不満足な顧客が回答しない傾向）
satisfaction_missing_prob = 0.1 + 0.2 * (customer_business_data['満足度'] < 3)
satisfaction_missing_mask = np.random.random(n_customers) < satisfaction_missing_prob
customer_business_data.loc[satisfaction_missing_mask, '満足度'] = np.nan

# 3. 購買頻度の欠損（ランダム）
freq_missing_mask = np.random.random(n_customers) < 0.05
customer_business_data.loc[freq_missing_mask, '購買頻度'] = np.nan

print("ビジネスケース：顧客購買データ")
print(f"データサイズ: {customer_business_data.shape}")
print("\n欠損値の状況:")
print(customer_business_data.isnull().sum())
print("\n欠損率（%）:")
print((customer_business_data.isnull().sum() / len(customer_business_data) * 100).round(2))

print("\nデータの概要:")
print(customer_business_data.describe())

### 7.1 ビジネスロジックに基づく包括的な前処理

In [ ]:
# 包括的なデータ前処理パイプライン
def comprehensive_preprocessing(df):
    """ビジネス要件を考慮した包括的な前処理"""
    
    processed_df = df.copy()
    preprocessing_log = []
    
    # 1. データ型の最適化
    # 年齢: int8で十分
    processed_df['年齢'] = processed_df['年齢'].astype('int8')
    
    # 性別と地域: category型
    processed_df['性別'] = processed_df['性別'].astype('category')
    processed_df['地域'] = processed_df['地域'].astype('category')
    processed_df['収入レベル'] = processed_df['収入レベル'].astype('category')
    
    # 年収: float32で十分
    processed_df['年収'] = processed_df['年収'].astype('float32')
    
    preprocessing_log.append("データ型を最適化")
    
    # 2. 年収の欠損値処理
    # 年齢・性別・地域グループごとの中央値で補完
    def fill_income(row):
        if pd.isna(row['年収']):
            # 同じ属性グループの中央値を使用
            mask = (
                (processed_df['年齢'] >= row['年齢'] - 5) & 
                (processed_df['年齢'] <= row['年齢'] + 5) &
                (processed_df['性別'] == row['性別']) &
                (processed_df['地域'] == row['地域'])
            )
            
            group_incomes = processed_df.loc[mask, '年収'].dropna()
            
            if len(group_incomes) >= 5:  # 十分なサンプルがある場合
                return group_incomes.median()
            else:  # サンプルが少ない場合は収入レベルの中央値を使用
                level_incomes = processed_df[
                    processed_df['収入レベル'] == row['収入レベル']
                ]['年収'].dropna()
                return level_incomes.median() if len(level_incomes) > 0 else processed_df['年収'].median()
        return row['年収']
    
    processed_df['年収'] = processed_df.apply(fill_income, axis=1)
    preprocessing_log.append("年収の欠損値を同属性グループの中央値で補完")
    
    # 3. 購買頻度の欠損値処理
    # 年収と年齢の関係から予測
    freq_complete_mask = processed_df['購買頻度'].notna()
    if freq_complete_mask.sum() > 100:  # 十分なデータがある場合
        from sklearn.linear_model import LinearRegression
        
        # 特徴量の準備
        X_complete = processed_df.loc[freq_complete_mask, ['年収', '年齢']]
        y_complete = processed_df.loc[freq_complete_mask, '購買頻度']
        
        # 回帰モデルの訓練
        freq_model = LinearRegression()
        freq_model.fit(X_complete, y_complete)
        
        # 欠損値の予測
        freq_missing_mask = processed_df['購買頻度'].isna()
        X_missing = processed_df.loc[freq_missing_mask, ['年収', '年齢']]
        
        predicted_freq = freq_model.predict(X_missing)
        processed_df.loc[freq_missing_mask, '購買頻度'] = predicted_freq
        
        preprocessing_log.append("購買頻度の欠損値を線形回帰で予測")
    else:
        # データが少ない場合は中央値で補完
        processed_df['購買頻度'] = processed_df['購買頻度'].fillna(
            processed_df['購買頻度'].median()
        )
        preprocessing_log.append("購買頻度の欠損値を中央値で補完")
    
    # 4. 満足度の欠損値処理
    # 購買頻度との関係を考慮
    satisfaction_median_by_freq = processed_df.groupby(
        pd.cut(processed_df['購買頻度'], bins=5)
    )['満足度'].median()
    
    def fill_satisfaction(row):
        if pd.isna(row['満足度']):
            freq_bin = pd.cut([row['購買頻度']], bins=5)[0]
            if freq_bin in satisfaction_median_by_freq.index:
                return satisfaction_median_by_freq[freq_bin]
            else:
                return processed_df['満足度'].median()
        return row['満足度']
    
    processed_df['満足度'] = processed_df.apply(fill_satisfaction, axis=1)
    preprocessing_log.append("満足度の欠損値を購買頻度レベル別中央値で補完")
    
    # 5. 外れ値の処理
    # 年収の外れ値をクリッピング
    income_q99 = processed_df['年収'].quantile(0.99)
    processed_df['年収'] = processed_df['年収'].clip(upper=income_q99)
    preprocessing_log.append(f"年収の外れ値を{income_q99:.0f}万円でクリッピング")
    
    # 6. 新しい特徴量の作成
    processed_df['年収_年齢比'] = processed_df['年収'] / processed_df['年齢']
    processed_df['購買_満足度'] = processed_df['購買頻度'] * processed_df['満足度']
    preprocessing_log.append("新しい特徴量を作成")
    
    return processed_df, preprocessing_log

# 前処理の実行
processed_data, log = comprehensive_preprocessing(customer_business_data)

print("前処理の実行ログ:")
for i, step in enumerate(log, 1):
    print(f"{i}. {step}")

print("\n前処理後の欠損値:")
print(processed_data.isnull().sum())

print("\n前処理後のデータ型:")
print(processed_data.dtypes)

print("\nメモリ使用量の比較:")
original_memory = customer_business_data.memory_usage(deep=True).sum() / 1024**2
processed_memory = processed_data.memory_usage(deep=True).sum() / 1024**2
print(f"元データ: {original_memory:.2f} MB")
print(f"処理後: {processed_memory:.2f} MB")
print(f"削減率: {(1 - processed_memory/original_memory)*100:.1f}%")

### 7.2 前処理結果の検証

In [ ]:
# 前処理の効果を検証
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. 年収分布の比較
axes[0, 0].hist(customer_business_data['年収'].dropna(), bins=30, alpha=0.7, 
               label='処理前', density=True)
axes[0, 0].hist(processed_data['年収'], bins=30, alpha=0.7, 
               label='処理後', density=True)
axes[0, 0].set_title('年収分布の比較')
axes[0, 0].legend()

# 2. 満足度分布の比較
axes[0, 1].hist(customer_business_data['満足度'].dropna(), bins=20, alpha=0.7, 
               label='処理前', density=True)
axes[0, 1].hist(processed_data['満足度'], bins=20, alpha=0.7, 
               label='処理後', density=True)
axes[0, 1].set_title('満足度分布の比較')
axes[0, 1].legend()

# 3. 購買頻度分布の比較
axes[0, 2].hist(customer_business_data['購買頻度'].dropna(), bins=30, alpha=0.7, 
               label='処理前', density=True)
axes[0, 2].hist(processed_data['購買頻度'], bins=30, alpha=0.7, 
               label='処理後', density=True)
axes[0, 2].set_title('購買頻度分布の比較')
axes[0, 2].legend()

# 4. 相関関係の比較
original_corr = customer_business_data.select_dtypes(include=[np.number]).corr()
processed_corr = processed_data.select_dtypes(include=[np.number]).corr()

im1 = axes[1, 0].imshow(original_corr, cmap='coolwarm', vmin=-1, vmax=1)
axes[1, 0].set_title('処理前の相関マトリックス')
axes[1, 0].set_xticks(range(len(original_corr.columns)))
axes[1, 0].set_yticks(range(len(original_corr.columns)))
axes[1, 0].set_xticklabels(original_corr.columns, rotation=45)
axes[1, 0].set_yticklabels(original_corr.columns)

im2 = axes[1, 1].imshow(processed_corr, cmap='coolwarm', vmin=-1, vmax=1)
axes[1, 1].set_title('処理後の相関マトリックス')
axes[1, 1].set_xticks(range(len(processed_corr.columns)))
axes[1, 1].set_yticks(range(len(processed_corr.columns)))
axes[1, 1].set_xticklabels(processed_corr.columns, rotation=45)
axes[1, 1].set_yticklabels(processed_corr.columns)

# 5. 欠損値パターンの変化
missing_before = customer_business_data.isnull().sum()
missing_after = processed_data.isnull().sum()

x = range(len(missing_before))
width = 0.35

axes[1, 2].bar([i - width/2 for i in x], missing_before, width, 
              label='処理前', alpha=0.7)
axes[1, 2].bar([i + width/2 for i in x], missing_after, width, 
              label='処理後', alpha=0.7)
axes[1, 2].set_title('欠損値数の変化')
axes[1, 2].set_xlabel('列')
axes[1, 2].set_ylabel('欠損値数')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(missing_before.index, rotation=45)
axes[1, 2].legend()

plt.tight_layout()
plt.show()

# 統計的検証
print("\n前処理の統計的検証:")
print("\n1. データ完全性:")
completeness_before = (1 - customer_business_data.isnull().sum().sum() / 
                      (len(customer_business_data) * len(customer_business_data.columns))) * 100
completeness_after = (1 - processed_data.isnull().sum().sum() / 
                     (len(processed_data) * len(processed_data.columns))) * 100
print(f"処理前: {completeness_before:.1f}%")
print(f"処理後: {completeness_after:.1f}%")

print("\n2. 主要な相関関係の保持:")
original_income_age_corr = customer_business_data[['年収', '年齢']].corr().iloc[0, 1]
processed_income_age_corr = processed_data[['年収', '年齢']].corr().iloc[0, 1]
print(f"年収-年齢相関 処理前: {original_income_age_corr:.3f}")
print(f"年収-年齢相関 処理後: {processed_income_age_corr:.3f}")

print("\n3. 新しい特徴量の有効性:")
new_feature_corr = processed_data[['購買_満足度', '購買頻度', '満足度']].corr()
print(f"購買_満足度と購買頻度の相関: {new_feature_corr.loc['購買_満足度', '購買頻度']:.3f}")
print(f"購買_満足度と満足度の相関: {new_feature_corr.loc['購買_満足度', '満足度']:.3f}")

## 8. まとめと実践的なガイドライン

### 今日学習した高度な技術
1. **高度な補完手法**
   - K近傍法（KNN）補完
   - 回帰による反復補完
   - 時系列データの季節性を考慮した補完

2. **欠損値パターンの分析**
   - MCAR/MAR/MNARの識別
   - 欠損パターンの可視化
   - 統計的検定による欠損機構の評価

3. **パフォーマンス最適化**
   - メモリ効率的なデータ型選択
   - 処理速度の比較評価
   - 大規模データでの実用的な手法選択

### 実践的なガイドライン

In [ ]:
# 実践的な欠損値処理のガイドライン
print("\n📋 実践的な欠損値処理ガイドライン")
print("="*50)

guidelines = {
    "1. 事前分析": [
        "• 欠損値の割合と分布を確認",
        "• 欠損パターンの可視化",
        "• ビジネス要件との照合"
    ],
    "2. 欠損機構の判定": [
        "• MCAR: ランダム削除 or 単純補完",
        "• MAR: 高度な補完手法",
        "• MNAR: ドメイン知識活用"
    ],
    "3. 手法選択の基準": [
        "• データサイズ: 小→単純、大→高度",
        "• 欠損率: <5%→削除、5-20%→補完、>20%→慎重検討",
        "• 計算資源: 制限有→高速手法、無制限→高精度手法"
    ],
    "4. 手法別推奨シナリオ": [
        "• 平均値/中央値: 単純な数値データ",
        "• KNN: 相関の強いデータ",
        "• 回帰: 高精度が必要な場合",
        "• 時系列補間: 連続的な時系列データ"
    ],
    "5. 検証ポイント": [
        "• 補完前後の分布比較",
        "• 相関関係の保持確認",
        "• 外れ値の発生確認",
        "• ビジネス指標への影響評価"
    ]
}

for category, items in guidelines.items():
    print(f"\n{category}")
    for item in items:
        print(f"  {item}")

print("\n\n🎯 最適な手法選択のフローチャート")
print("="*50)
flowchart = """
データの欠損値を発見
    ↓
欠損率 < 5% ?
    ↓ Yes → 行削除を検討
    ↓ No
計算資源に制約あり?
    ↓ Yes → 単純補完（平均値/中央値/最頻値）
    ↓ No
データに相関関係あり?
    ↓ Yes → KNN補完 or 回帰補完
    ↓ No
時系列データ?
    ↓ Yes → 線形補間 or 季節性考慮補完
    ↓ No
カテゴリカルデータ?
    ↓ Yes → 最頻値 or 新カテゴリ作成
    ↓ No
ビジネスルールの適用
"""
print(flowchart)

print("\n\n⚠️ 注意すべきポイント")
print("="*50)
warnings_list = [
    "1. 補完は情報を『創造』するのではなく『推定』する",
    "2. 過度な補完は過学習の原因となる",
    "3. 補完結果は必ず検証・妥当性確認を行う",
    "4. ビジネス要件に反する補完は避ける",
    "5. 補完前のデータも保持し、トレーサビリティを確保"
]

for warning in warnings_list:
    print(f"  {warning}")

## 9. 次のステップ

### 学習の発展方向
- **高度な機械学習手法**: Deep Learning based imputation
- **時系列の特殊処理**: ARIMA, Prophet, LSTMベース補完
- **大規模データ処理**: Dask, Apache Spark
- **自動化パイプライン**: MLOpsでの前処理自動化

### 実践課題
1. Kaggleコンペティションデータでの前処理実践
2. 自社データでの欠損値パターン分析
3. A/Bテストでの補完手法効果検証
4. リアルタイムデータストリームでの欠損値処理